# Investment Comp Screener

Premia/ Discount (P/D)

Sector 3 year Z Score (S3ZS)

Company 3 year Z Score (C3ZS)

In [1]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm
import os
import glob
import re
import seaborn as sns

from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
def calc_ecdf(data):
    #low is good
    ecdf= ECDF(data)
    current_perc = ecdf(data[-1])
    
    return(current_perc)

In [3]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

In [4]:
def Z_score(data):
    # not including the current observation
    std = data.iloc[:-1].std()
    mean = data.iloc[:-1].mean()
    Z = (data[-1:] - mean)/std
    return Z
    

In [5]:
def mod_Z_score(data):
    median = np.median(data.iloc[:-1])
    deviation_from_med = np.array(data.iloc[:-1]) - median
    MAD = np.median(np.abs(deviation_from_med))
    mod_Z = (data[-1:] - median)/(MAD * 1.4826)
    return mod_Z

## Data Import

In [6]:
path = "holding_companies_data"
arr = [x for x in os.listdir(path) if x.endswith(".csv")]

In [7]:
for f in arr:
    print(f)

2021-02-20_lato b_pris_historik.csv
2021-02-20_ores_pris_historik.csv
2021-02-20_vnv_pris_historik.csv
2021-02-20_kinv b_pris_historik.csv
2021-02-20_inve b_pris_historik.csv
2021-02-20_bure_pris_historik.csv
2021-02-20_cred a_pris_historik.csv
2021-02-20_svol b_pris_historik.csv
2021-02-20_indu c_pris_historik.csv
2021-02-20_lund b_pris_historik.csv
2021-02-20_trac b_pris_historik.csv


In [8]:
for k in arr:
    if "trac" in k:
        traction = pd.read_csv(os.path.join(path,k))
    if "lund" in k:
        lundbergs = pd.read_csv(os.path.join(path,k))
    if "hav" in k:
        havsfrun = pd.read_csv(os.path.join(path,k))
    if "ores" in k:
        oresund = pd.read_csv(os.path.join(path,k))
    if "indu" in k:
        industri = pd.read_csv(os.path.join(path,k))
    if "svol" in k:
        svolder = pd.read_csv(os.path.join(path,k))
    if "bure" in k:
        bure = pd.read_csv(os.path.join(path,k))
    if "cred" in k:
        creades = pd.read_csv(os.path.join(path,k))
    if "inve" in k:
        investor = pd.read_csv(os.path.join(path,k))
    if "kinv" in k:
        kinnevik = pd.read_csv(os.path.join(path,k))
    if "vnv" in k:
        vnv = pd.read_csv(os.path.join(path,k))
    if "naxs" in k:
        naxs = pd.read_csv(os.path.join(path,k))
    if "lato" in k:
        latour = pd.read_csv(os.path.join(path,k))
    

## Calculate Metrics

In [9]:
look_back = 3*252 #3 years
ST_look_back = 5


#companies = [traction, lundbergs, havsfrun, oresund, industri,
#             svolder, bure, creades, investor, kinnevik, vnv, naxs,latour]

#companies_names = ['Traction', 'Lundbergsföretagen', 'Havsfrun Investment', 'Öresund', 'Industrivärden',
#            'Svolder', 'Bure', 'Creades', 'Investor', 'Kinnevik', 'VNV Global', 'NAXS','Latour']

companies = [traction, lundbergs, oresund, industri,
             svolder, bure, creades, investor, kinnevik, vnv,latour]

companies_names = ['Traction', 'Lundbergsföretagen', 'Öresund', 'Industrivärden',
             'Svolder', 'Bure', 'Creades', 'Investor', 'Kinnevik', 'VNV Global','Latour']


for comp in companies:
    
    comp['DATUM'] = pd.to_datetime(comp['DATUM'])
    comp.set_index(['DATUM'], inplace=True, drop=True)
    comp['P/D']=comp['PRIS']/comp['BERÄKNAT_SUBSTANSVÄRDE'] - 1
    



In [10]:
#high DSSP and HIDI is good

AVG_DISC = pd.DataFrame(latour['PRIS'].copy())
AVG_DISC['PRIS'] = 0
AVG_DISC.columns = ['AVG_DISC']

for comp in companies:

    AVG_DISC['AVG_DISC'] = AVG_DISC['AVG_DISC'] + comp['P/D']

AVG_DISC = AVG_DISC/len(companies)

for comp in companies:
    comp['SPREAD'] = comp['P/D'] - AVG_DISC['AVG_DISC']
    #comp['DSSP'] = -comp['SPREAD'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['S3ZS'] = -comp['SPREAD'].rolling(look_back).apply(lambda x: Z_score(x),raw=False) 
    #comp['HIDI'] = -comp['R/P'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['C3ZS'] = -comp['P/D'].rolling(look_back).apply(lambda x: Z_score(x),raw=False)
    comp['MOM'] = (comp['PRIS'].pct_change(252) + comp['PRIS'].pct_change(126) + comp['PRIS'].pct_change(63))/3
    comp['NAV MOM'] = (comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(252) + comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(126) + comp['BERÄKNAT_SUBSTANSVÄRDE'].pct_change(63))/3
    comp['ST'] = comp['PRIS'].pct_change(ST_look_back)
    
    

In [11]:
INV_COMP = None

for comp in companies:
    
    INV_COMP = pd.concat([INV_COMP, comp.tail(1)], axis = 0)

INV_COMP['Company'] = companies_names
INV_COMP.set_index(['Company'], inplace=True, drop=True)



In [12]:
INV_COMP['VRANK'] = (INV_COMP['P/D'].rank() + INV_COMP['S3ZS'].rank(ascending = False) +
 INV_COMP['C3ZS'].rank(ascending = False)).rank()



INV_COMP['MRANK'] = (INV_COMP['MOM'].rank(ascending = False)+INV_COMP['NAV MOM'].rank(ascending = False)).rank()

INV_COMP['CRANK'] = (INV_COMP['VRANK'] + INV_COMP['MRANK']).rank()


compact = ['P/D', 'C3ZS', 'S3ZS', 'MOM', 'NAV MOM', 'VRANK', 'MRANK', 'CRANK', 'ST']

INV_COMP = INV_COMP[compact]

INV_COMP['P/D']=-INV_COMP['P/D']

In [13]:
average = pd.DataFrame(INV_COMP.mean(axis = 0)).transpose()
average.index = ['Average']
INV_COMP = pd.concat([INV_COMP, average], axis = 0)

## Display Results

In [14]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)

INV_COMP_VIEW = (INV_COMP.style
    .applymap(color_negative_red, subset=['P/D', 'MOM', 'NAV MOM'])
    .background_gradient(axis = 0 ,subset = ['S3ZS', 'C3ZS'],cmap=cm_high_good)
    .background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','CRANK', 'ST'],cmap=cm_low_good)
    #.set_caption('Performance Metrics')
    .format({'P/D': "{:.1%}", 'S3ZS': "{:.2f}", 'C3ZS': "{:.2f}", 'MOM':"{:.1%}", 'NAV MOM': "{:.1%}",
            'Sortino': "{:.2f}", 'MAR': "{:.2f}", 'VRANK': "{:.0f}", 'MRANK': "{:.0f}", 'CRANK': "{:.0f}",'ST':"{:.1%}"})
    .set_table_styles(styles))

In [15]:
INV_COMP_VIEW

,P/D,C3ZS,S3ZS,MOM,NAV MOM,VRANK,MRANK,CRANK,ST
Traction,8.0%,0.39,1.17,7.9%,10.4%,2,10,6,-0.8%
Lundbergsföretagen,5.4%,0.79,1.63,-1.7%,13.8%,2,10,6,-2.5%
Öresund,0.5%,0.49,0.92,1.4%,3.2%,4,11,9,-2.5%
Industrivärden,7.8%,-1.05,0.40,16.6%,15.7%,6,6,7,-0.1%
Svolder,4.2%,-0.33,1.06,26.2%,34.6%,5,4,2,-2.1%
Bure,-21.6%,-1.43,-1.76,33.7%,28.6%,10,4,8,-0.5%
Creades,-19.1%,-0.02,0.30,20.3%,53.2%,8,2,4,-6.4%
Investor,18.4%,-0.60,0.75,15.2%,16.8%,3,6,2,-2.6%
Kinnevik,7.4%,-0.88,-0.09,43.1%,39.0%,7,1,1,-6.3%
VNV Global,-12.6%,-2.07,-1.91,35.3%,2.4%,10,7,11,-3.3%
